In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall -y kaggle
!pip install kaggle
import os
import zipfile

os.environ["KAGGLE_USERNAME"] = "mizanurr"
os.environ["KAGGLE_KEY"] = "654c2a7dc145088aa93c65566a027cf6"
!kaggle competitions download -c stanford-covid-vaccine
!kaggle datasets download -d vaghefi/aug-vaccine


Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
     |████████████████████████████████| 61kB 2.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-cp36-none-any.whl size=73275 sha256=4e52e70aacd8611528e52a9332fd375f54b493b5f9d06fac0a061d4cd9da3098
  Stored in directory: /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c
Successfully built kaggle
 45% 20.0M/44.1M [00:00<00:00, 38.3MB/s]
100% 44.1M/44.1M [00:00<00:00, 128MB/s] 
 83% 81.0M/97.1M [00:00<00:00, 111MB/s] 
100% 97.1M/97.1M [00:00<00:00, 121MB/s]


In [ ]:
!mkdir data

In [ ]:
with zipfile.ZipFile('/content/stanford-covid-vaccine.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

In [ ]:
with zipfile.ZipFile('/content/aug-vaccine.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

In [ ]:
!pip install spektral

     |████████████████████████████████| 102kB 4.1MB/s 


In [ ]:
#the basics
import pandas as pd 
import numpy as np
import seaborn as sns
import math, json
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
import time
from spektral.layers import GraphAttention, GatedGraphConv

#tensorflow deep learning basics
import tensorflow as tf

print('tf version:',tf.__version__)

print('\nList of GPUs:')
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

tf version: 2.3.0

List of GPUs:
Name: /physical_device:GPU:0   Type: GPU


In [ ]:
start = time.time()

import os
folder = '/content/data/bpps'
files = os.listdir(folder)

lst = []
for file in tqdm(files):
    data = np.load(os.path.join(folder, file))
    lst.append([file.replace('.npy',''), data])
    
bpps = pd.DataFrame(lst, columns=['id', 'bpps'])

100%|██████████| 6034/6034 [00:02<00:00, 2200.57it/s]


In [ ]:
def get_structure_feature(structure):
    pm = np.zeros((len(structure), 3))
    start_token_indices = []
    for i, token in enumerate(structure):
        if token == "(":
            start_token_indices.append(i)
        elif token == ")":
            j = start_token_indices.pop()
            pm[i, 0] = pm[i, 0] = i - j
            pm[i, 1] = pm[j, 1] = structure[j:i+1].count('.')
            pm[i, 2] = pm[j, 2] = structure[j:i+1].count(')') - 1
    return pm

In [ ]:
train = pd.read_json('/content/data/train.json', lines=True).merge(bpps)
test = pd.read_json('/content/data/test.json', lines=True).merge(bpps)
sample_sub = pd.read_csv('/content/data/sample_submission.csv')
augment = pd.read_csv('/content/drive/My Drive/kaggle/aug_data1.csv')

In [ ]:
# train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True).merge(bpps)
# test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True).merge(bpps)
# sample_sub = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')
# augment = pd.read_csv('../input/aug-vaccine/aug_data1.csv')


train_aug = train.drop(columns=['structure', 'predicted_loop_type']).merge(augment, how='left', on=['id','sequence'])
test_aug = test.drop(columns=['structure', 'predicted_loop_type']).merge(augment, how='left', on=['id','sequence'])

test = test.append(test_aug[test.columns])

#target columns
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C']
metric_cols = [0, 1, 3]

token = {'sequence': 'ACGU', 'structure': '().', 'predicted_loop_type': 'BEHIMSX'}
token_map = {key: {x:i for i, x in enumerate(val)} for key, val in token.items()}

def get_adj(structure):
    pm = np.zeros((len(structure), len(structure)))
    start_token_indices = []
    for i, token in enumerate(structure):
        if token == "(":
            start_token_indices.append(i)
        elif token == ")":
            j = start_token_indices.pop()
            pm[i, j] = 1.0
            pm[j, i] = 1.0
        pm[i, i] = 1
    return pm


mu_bpps = 0.368
sd_bpps = 0.415

def preprocess_inputs(df):
    features = []
    for col in ['sequence', 'structure', 'predicted_loop_type']:
        x = np.array(df[col].apply(lambda seq: [token_map[col][x] for x in seq]).values.tolist())
        features.append(tf.keras.backend.one_hot(x,len(token_map[col])))
    bpps = np.array(df['bpps'].tolist())
    features.append((1-bpps.sum(-1))[:,:,None])
    features.append(bpps.max(-1)[:,:,None])
    features.append((bpps > 0).mean(-1)[:,:,None])
    features.append(np.array(df['structure'].apply(get_structure_feature).values.tolist())/130)
    return np.concatenate(features, axis=-1)

print('processing train')
train_snfilter = train.SN_filter.values
train_inputs = preprocess_inputs(train)
train_inputs = np.pad(train_inputs,((0, 0), (0, 130-107), (0, 0)))

train_inputs_aug = preprocess_inputs(train_aug)
train_inputs_aug = np.pad(train_inputs_aug,((0, 0), (0, 130-107), (0, 0)))
train_ids_aug = train_aug['id'].values

train_weights = np.array(train.signal_to_noise.values.tolist())
train_weights_aug = np.array(train_aug.signal_to_noise.values.tolist())

train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))
train_labels_aug = np.array(train_aug[target_cols].values.tolist()).transpose((0, 2, 1))

print('Original Train:', train_inputs.shape, train_labels.shape, train_weights.shape)
print('Augmented Train:', train_inputs_aug.shape, train_labels_aug.shape, train_weights_aug.shape)

train_adj = np.array(train['structure'].apply(get_adj).values.tolist())
train_adj = np.pad(train_adj, ((0,0),(0,130-107),(0,130-107)), 'constant')
train_adj_aug = np.array(train_aug['structure'].apply(get_adj).values.tolist())
train_adj_aug = np.pad(train_adj_aug, ((0,0),(0,130-107),(0,130-107)), 'constant')

processing train
Original Train: (2400, 130, 20) (2400, 68, 5) (2400,)
Augmented Train: (2400, 130, 20) (2400, 68, 5) (2400,)


In [ ]:
#get different test sets and process each
public_df = test.query("seq_length == 107").copy()
private_df = test.query("seq_length == 130").copy()

print('processing test')
public_inputs = preprocess_inputs(public_df)
public_inputs = np.pad(public_inputs,((0, 0), (0, 130-public_inputs.shape[1]), (0, 0)))

private_inputs = preprocess_inputs(private_df)
private_inputs = np.pad(private_inputs,((0, 0), (0, 130-private_inputs.shape[1]), (0, 0)))

public_adj = np.array(public_df.loc[:, 'structure'].apply(get_adj).values.tolist())
public_adj = np.pad(public_adj, ((0,0),(0,130-public_adj.shape[1]),(0,130-public_adj.shape[1])), 'constant')

private_adj = np.array(private_df.loc[:, 'structure'].apply(get_adj).values.tolist())
private_adj = np.pad(private_adj, ((0,0),(0,130-private_adj.shape[1]),(0,130-private_adj.shape[1])), 'constant')

print('Test Sizes:', public_inputs.shape, public_adj.shape, private_inputs.shape, private_adj.shape)


processing test
Test Sizes: (1258, 130, 20) (1258, 130, 130) (6010, 130, 20) (6010, 130, 130)


In [ ]:
public_preds = np.load('/content/data/gtn_aug_snr_gru_public_preds.npy')
public_std = public_preds.mean(3).mean(2).std(0)
good_public = np.where(public_std < np.quantile(public_std, 0.13))[0]
public_labels = public_preds.mean(0)[good_public][:,:68,:]
print('Public Pseudo-labeling Sizes:', public_labels.shape)

Public Pseudo-labeling Sizes: (164, 68, 5)


In [ ]:
private_preds = np.load('/content/data/gtn_aug_snr_gru_private_preds.npy')
private_std = private_preds.mean(3).mean(2).std(0)
good_private = np.where(private_std < np.quantile(private_std, 0.13))[0]
private_labels = private_preds.mean(0)[good_private][:,:68,:]
print('Private Pseudo-labeling Sizes:', private_labels.shape)

Private Pseudo-labeling Sizes: (782, 68, 5)


In [ ]:
nnodes = train_inputs.shape[-2]          # Number of nodes in the graphs
nfeatures = train_inputs.shape[-1]          # Original feature dimensionality

def cbr(x, out_layer, kernel, stride, dilation):
        x = L.Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        # x = L.BatchNormalization()(x)
        x = L.Activation("relu")(x)
        return x

def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = tf.keras.layers.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = tf.keras.layers.Multiply()([tanh_out, sigm_out])
            x = tf.keras.layers.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = tf.keras.layers.Add()([res_x, x])
        return res_x

def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.GRU(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def mcrmse_loss(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def mcrmse(y_true, y_pred):
    diff = tf.reshape(y_true[:,:,:5] - y_pred, [-1, 5])
    colwise_mse = tf.reduce_mean(tf.square(diff), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse))


def competition_metric(y_true, y_pred):
    total = 0
    for i in range(5):
        rmse = np.mean((y_pred[:,:,i] - y_true[:,:,i])**2)**0.5
        print('{:12s}: {:.4f}'.format(target_cols[i], rmse))
        if target_cols[i] in ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']:
            total += rmse/3
    print('total:', total)

def build_model(model_num=0,seq_len=130, pred_len=68, dropout=0.4,
                embed_dim=512, hidden_dim=256):
    
    X_in = tf.keras.layers.Input(shape=(seq_len, nfeatures))
    A_in = tf.keras.layers.Input(shape=(seq_len, seq_len))
    embed = tf.keras.layers.Dense(embed_dim, activation='linear')(X_in)    
        
    gc1 = GraphAttention(embed_dim, activation='relu', dropout_rate=0.1)([X_in, A_in])
    gc1 = tf.keras.layers.Dropout(.2)(gc1)
    gc1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc1 + embed)
    
    gc2 = GraphAttention(embed_dim, activation='relu', dropout_rate=0.1)([gc1, A_in])
    gc2 = tf.keras.layers.Dropout(.2)(gc2)
    gc2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc2 + gc1)
    
    reshaped = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc2 + embed)
    
    
    if model_num == 0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)

    if model_num == 1:
        hidden =cbr(reshaped, 64, 7, 1, 1)
        hidden = cbr(hidden, 64, 7, 1, 1)#
        hidden = cbr(hidden, 64, 7, 1, 1)   
              
    if model_num == 2:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)

    if model_num == 3:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)

    if model_num == 4:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)

    if model_num == 5:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden) 
        
    if model_num == 6:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        
    if model_num == 7:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
    
    hidden = wave_block(hidden, 16, 3, 12)
    hidden = wave_block(hidden, 32, 3, 8)
    hidden = wave_block(hidden, 64, 3, 4)
    hidden = wave_block(hidden, 128, 3, 1)
    
    #only making predictions on the first part of each sequence
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=[X_in, A_in], outputs=out)

    #some optimizers
    adam = tf.optimizers.Adam()
    
    model.compile(optimizer = adam, loss=mcrmse_loss)
    
    return model

In [ ]:
#basic training configuration
FOLDS = 10
EPOCHS = 500
BATCH_SIZE = 64*len(gpus)
VERBOSE = 1

lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=VERBOSE, factor=0.9, patience = 5 )
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=VERBOSE, mode='auto')

In [ ]:
import tensorflow.keras.layers as L
import keras.backend as K
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.cluster import KMeans

In [ ]:
kfold = KFold(FOLDS, shuffle = True, random_state = 34)

#strategy = tf.distribute.MirroredStrategy()

model_name = 'gtn_aug_snr'

train_ids = train['id'].values

models = [0,1,2,3]

gru_private_preds = [None]*len(models)
gru_public_preds = [None]*len(models)
oof = [None]*len(models)

for model_num in models:
    
    gru_private_preds[model_num] = np.zeros((private_df.shape[0], 130, 5))
    gru_public_preds[model_num] = np.zeros((public_df.shape[0], 107, 5))
    oof[model_num] = np.zeros((train_labels.shape[0], 68, 5))

    for fold, (train_index, val_index) in enumerate(list(kfold.split(train_inputs))[:]):
        print('Fold:', fold, ' model_num:', model_num)
        cp = tf.keras.callbacks.ModelCheckpoint(f'{model_name}_{fold}_{model_num}.h5', monitor='val_loss', verbose=VERBOSE, save_best_only=True,
            save_weights_only=True, mode='auto', save_freq='epoch')
        
        #train_index = train_index[train_snfilter[train_index] == 1]
        train_X = train_inputs[train_index]
        train_y = train_labels[train_index]
        train_A = train_adj[train_index]
        train_W = train_weights[train_index]
        
        if 1:
            aug_indx = np.isin(train_ids_aug, train_ids[train_index])    
            train_X_aug = train_inputs_aug[aug_indx]
            train_y_aug = train_labels_aug[aug_indx]
            train_A_aug = train_adj_aug[aug_indx]
            train_W_aug = train_weights_aug[aug_indx]

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug))
        
        if 0:
            train_X_aug = public_inputs[good_public]
            train_A_aug = public_adj[good_public]
            train_W_aug = train_weights[train_index].mean()*np.ones(train_X_aug.shape[0])
            train_y_aug = public_labels

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug))    
            
            train_X_aug = private_inputs[good_private]
            train_A_aug = private_adj[good_private]
            train_W_aug = train_weights[train_index].mean()*np.ones(train_X_aug.shape[0])
            train_y_aug = private_labels

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug)) 
        
        print(train_X.shape)
        
        val_index = val_index[train_snfilter[val_index] == 1]
        valid_X = train_inputs[val_index]
        valid_y = train_labels[val_index]
        valid_A = train_adj[val_index]
        
        model = build_model(model_num)
        history = model.fit([train_X,train_A], train_y, sample_weight = train_W, validation_data=([valid_X,valid_A],valid_y), 
                            batch_size=BATCH_SIZE, epochs=EPOCHS,callbacks=[lr,cp,es],verbose = VERBOSE)      
        
        model.load_weights(f'{model_name}_{fold}_{model_num}.h5')
        oof[model_num][val_index] = model.predict([valid_X,valid_A])
        
        competition_metric(valid_y, oof[model_num][val_index])
    
        
        if 1:
            #load best model and predict
            gru_short = build_model(model_num, pred_len=107)
            gru_short.load_weights(f'{model_name}_{fold}_{model_num}.h5')
            gru_public_pred = gru_short.predict([public_inputs,public_adj]) / FOLDS
            
            gru_long = build_model(model_num, pred_len=130)
            gru_long.load_weights(f'{model_name}_{fold}_{model_num}.h5')
            gru_private_pred = gru_long.predict([private_inputs, private_adj]) / FOLDS
            
            gru_public_preds[model_num] += gru_public_pred
            gru_private_preds[model_num] += gru_private_pred
      
    print('===================OOF RMSE=================')
    competition_metric(train_labels[train_snfilter == 1], oof[model_num][train_snfilter == 1])
    print('============================================')


print('===================COMBINED OOF RMSE=================')
competition_metric(train_labels[train_snfilter == 1], np.mean(oof, axis=0)[train_snfilter == 1])
print('=====================================================')

Fold: 0  model_num: 0
(4320, 130, 20)
Epoch 1/500
68/68 [==============================] - ETA: 0s - loss: 2.2770
Epoch 00001: val_loss improved from inf to 0.32976, saving model to gtn_aug_snr_0_0.h5
68/68 [==============================] - 24s 359ms/step - loss: 2.2770 - val_loss: 0.3298
Epoch 2/500
68/68 [==============================] - ETA: 0s - loss: 1.4934
Epoch 00002: val_loss improved from 0.32976 to 0.31037, saving model to gtn_aug_snr_0_0.h5
68/68 [==============================] - 19s 282ms/step - loss: 1.4934 - val_loss: 0.3104
Epoch 3/500
68/68 [==============================] - ETA: 0s - loss: 1.3840
Epoch 00003: val_loss improved from 0.31037 to 0.28511, saving model to gtn_aug_snr_0_0.h5
68/68 [==============================] - 19s 283ms/step - loss: 1.3840 - val_loss: 0.2851
Epoch 4/500
68/68 [==============================] - ETA: 0s - loss: 1.3138
Epoch 00004: val_loss improved from 0.28511 to 0.27326, saving model to gtn_aug_snr_0_0.h5
68/68 [=====================

In [ ]:
if 0:
    #plt.plot(history.history['loss'][10:], label='train')
    plt.plot(history.history['val_loss'][10:], label='val')
    plt.yscale('log')
    plt.legend()

In [ ]:
np.save(f'/content/{model_name}_oof', oof)
np.save(f'/content/{model_name}_gru_public_preds', gru_public_preds)
np.save(f'/content/{model_name}_gru_private_preds', gru_private_preds)


#gru_public_preds = np.load(f'preds/{model_name}_gru_public_preds.npy')
#gru_private_preds = np.load(f'preds/{model_name}_gru_private_preds.npy')

gru_private_preds = np.mean(gru_private_preds, axis=0)
gru_public_preds = np.mean(gru_public_preds, axis=0)

preds_gru = []

for df, preds in [(public_df, gru_public_preds), (private_df, gru_private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=target_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_gru.append(single_df)

preds_gru_df = pd.concat(preds_gru)

preds_gru_df = preds_gru_df.groupby('id_seqpos').mean().reset_index()


submission = sample_sub[['id_seqpos']].merge(preds_gru_df, on=['id_seqpos'])

#sanity check
submission.head()

submission.to_csv(f'/content/drive/My Drive/kaggle/submissionwavenetfinalv1.csv', index=False)
print('Submission saved')

print('Run Time [min]:', round((time.time() - start)/60))